In [3]:
import pandas as pd
import statsmodels.formula.api as smf

df_6 = pd.read_csv(r'C:\Users\clint\Desktop\Research-Paper-Trucks\Exploration\5\df_6.csv')
df_6
# Load the dataset
df = df_6
df

,REPORT_DATE,FATALITIES,STATE,COUNTY,change_2007,change_2008,change_2015,change_2016,change_2006_2016,has_accident_before_2006,name,Longitude,Latitude,Region,Family,Color,Nearest Neighbor,Region_color,simple_family
0,2024-10-17,0,CA,6037,0.0,0.0,-2.0,0.0,-6.0,0,Los Angeles,-118.224703,34.321346,W,Growth Management,green,Los Angeles,#1f77b4,Reform
1,2024-11-17,0,CA,6019,0.0,-1.0,0.0,0.0,1.0,0,Fresno,-119.650683,36.758077,W,Growth Management,green,Sacramento,#1f77b4,Reform
2,2024-12-06,0,ID,16001,0.0,0.0,0.0,0.0,0.0,0,Ada,-116.241035,43.450841,W,High Density,orange,Salt Lake City,#1f77b4,Traditional
3,2024-11-22,0,ID,16001,0.0,0.0,0.0,0.0,0.0,0,Ada,-116.241035,43.450841,W,High Density,orange,Salt Lake City,#1f77b4,Traditional
4,2024-12-05,0,IL,17159,0.0,0.0,0.0,0.0,0.0,0,Richland,-88.085602,38.712287,MW,Middle America,orange,Cincinnati (IN),#d62728,Traditional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41590,1997-09-23,0,SD,46099,1.0,0.0,0.0,-1.0,-1.0,0,Minnehaha,-96.791455,43.674134,MW,Middle America,orange,Minneapolis (MN),#d62728,Traditional
41591,2000-07-16,0,SD,46093,2.0,0.0,0.0,1.0,5.0,0,Meade,-102.716646,44.566675,W,Growth Control,green,Denver,#1f77b4,Reform
41592,2001-02-03,0,SD,46099,1.0,0.0,0.0,-1.0,-1.0,0,Minnehaha,-96.791455,43.674134,MW,Middle America,orange,Minneapolis (MN),#d62728,Traditional
41593,1999-01-20,0,SD,46099,1.0,0.0,0.0,-1.0,-1.0,0,Minnehaha,-96.791455,43.674134,MW,Middle America,orange,Minneapolis (MN),#d62728,Traditional


In [4]:
unique_fatalities = df_6['FATALITIES'].unique()
print(unique_fatalities)

[0 1 2 3 7 4 5 6]


In [5]:

# Convert the accident report date to datetime and extract the accident year
df['REPORT_DATE'] = pd.to_datetime(df['REPORT_DATE'])
df['accident_year'] = df['REPORT_DATE'].dt.year

# Define an accident indicator (1 if FATALITIES >= 1, adjust as needed)
df['Accident'] = (df['FATALITIES'] >= 1).astype(int)

# --- Aggregate accident info by county ---
# For each county, if any accident occurred, mark Accident = 1.
accident_by_county = df.groupby(['STATE', 'COUNTY']).agg({'Accident': 'max'}).reset_index()

# --- Reshape the truck stop change data from wide to long format ---
truck_stop_cols = ['change_2007', 'change_2008', 'change_2015', 'change_2016']
truck_df = df[['STATE', 'COUNTY', 'Region', 'simple_family'] + truck_stop_cols].drop_duplicates()

# Melt the data to have a county-year structure
truck_df_long = truck_df.melt(
    id_vars=['STATE', 'COUNTY', 'Region', 'simple_family'],
    value_vars=truck_stop_cols,
    var_name='year',
    value_name='delta_NumTruckStop'
)

# Extract the year from column names
truck_df_long['year'] = truck_df_long['year'].str.extract(r'(\d{4})').astype(int)

# --- Merge accident data ---
merged_df = pd.merge(truck_df_long, accident_by_county, on=['STATE', 'COUNTY'], how='left')

# Create the "Post" indicator (1 if year >= 2016, 0 otherwise)
merged_df['Post'] = (merged_df['year'] >= 2016).astype(int)

# Create the interaction term: Accident * Post
merged_df['Accident_Post'] = merged_df['Accident'] * merged_df['Post']

# --- Run the regression with fixed effects ---
model = smf.ols(
    'delta_NumTruckStop ~ Accident_Post + C(STATE) + C(Region) + C(simple_family)',
    data=merged_df
).fit()

# Print results
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:     delta_NumTruckStop   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.466
Date:                Mon, 24 Mar 2025   Prob (F-statistic):             0.0141
Time:                        14:09:14   Log-Likelihood:                -4311.6
No. Observations:                7984   AIC:                             8735.
Df Residuals:                    7928   BIC:                             9126.
Df Model:                          55                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 